In [100]:
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RandomizedSearchCV
from pprint import pprint
from sklearn.ensemble import ExtraTreesRegressor
import numpy as np
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from dtreeviz.trees import dtreeviz

# If you're working in Jupyter Notebook, include the following so that plots will display:
%matplotlib inline

In [101]:
df = pd.read_csv('longitudinal_hoos_func_final.csv')
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

In [102]:
df.head()

,participant_id,age,gender,height_m,weight_kg,bmi,mhi5_score,mhi5_score_baseline,hoos_symptoms_score,hoos_adl_score,hoos_sports_score,hoos_qol_score,hoos_womac_pain,hoos_womac_stiffness,hoos_womac_function,hoos_pain_score,hoos_pain_score_baseline_arm_1,hoos_pain_score_1_yr_follow_up_arm_1,hoos_pain_score_2_yr_follow_up_arm_1,char_rise,m_walk_test,stair_climbing,side_plank,char_rise_baseline,m_walk_test_baseline,stair_climbing_baseline,side_plank_baseline,char_rise_1yr,m_walk_test_1yr,stair_climbing_1yr,side_plank_1yr,char_rise_2yr,m_walk_test_2yr,stair_climbing_2yr,side_plank_2yr,t1rho_mean_fem_whole_cartilage,t2_mean_fem_whole_cartilage,t1rho_mean_ace_whole_cartilage,t2_mean_ace_whole_cartilage,t1rho_mean_fem_whole_cartilage_baseline,t2_mean_fem_whole_cartilage_baseline,t1rho_mean_ace_whole_cartilage_baseline,t2_mean_ace_whole_cartilage_baseline,race___1,race___2,race___3,race___4,race___5,race___6,ethnicity___1,ethnicity___2,ethnicity___3
0,P002L,63,2,1.72,62.03,20.97,84,92,100,100.0,100.00,100.0,100,100.0,100.0,100.0,100.0,100.0,100.0,25,15.72,8.43,74.81,19,14.51,9.29,72.0,24,14.80,8.10,82.00,26,14.20,8.50,74.00,36.737011,39.808437,37.536424,36.280691,36.894112,36.274728,39.063521,36.883323,0,0,0,0,0,0,0,0,0
1,P002R,63,2,1.72,62.03,20.97,84,92,100,100.0,100.00,100.0,100,100.0,100.0,100.0,100.0,100.0,100.0,25,15.72,8.43,72.37,19,14.51,9.29,88.0,24,14.80,8.10,90.00,26,14.20,8.50,80.10,30.908057,27.011128,34.446011,30.539640,34.964481,26.479733,33.947857,28.195027,0,0,0,0,0,0,0,0,0
2,P006L,59,1,1.65,63.70,23.40,92,72,100,100.0,93.75,75.0,100,100.0,100.0,97.5,95.0,97.5,97.5,30,16.71,8.29,128.93,27,15.87,8.31,151.0,30,16.00,7.60,149.00,32,14.20,7.80,131.70,36.679127,38.343637,38.148536,38.226135,33.630381,32.819486,33.240301,30.177376,0,0,0,0,0,0,0,0,0
3,P006R,59,1,1.65,63.70,23.40,92,72,100,100.0,100.00,75.0,100,100.0,100.0,97.5,100.0,100.0,100.0,30,16.71,8.29,96.97,27,15.87,8.31,151.0,30,16.00,7.60,136.00,32,14.20,7.80,115.10,32.339383,27.341844,35.251564,29.626638,33.931334,25.997281,33.132494,27.046791,0,0,0,0,0,0,0,0,0
4,P008L,39,1,1.89,112.90,31.61,80,80,90,100.0,100.00,100.0,100,87.5,100.0,100.0,77.5,92.5,100.0,21,7.43,17.90,58.77,14,17.71,11.17,62.0,19,17.68,11.27,72.63,18,18.33,11.05,73.93,45.518854,41.328940,47.993043,39.459341,35.449060,34.427221,38.001963,32.516748,0,0,0,0,0,0,0,0,0


In [103]:
list(df.columns)

['participant_id',
 'age',
 'gender',
 'height_m',
 'weight_kg',
 'bmi',
 'mhi5_score',
 'mhi5_score_baseline',
 'hoos_symptoms_score',
 'hoos_adl_score',
 'hoos_sports_score',
 'hoos_qol_score',
 'hoos_womac_pain',
 'hoos_womac_stiffness',
 'hoos_womac_function',
 'hoos_pain_score',
 'hoos_pain_score_baseline_arm_1',
 'hoos_pain_score_1_yr_follow_up_arm_1',
 'hoos_pain_score_2_yr_follow_up_arm_1',
 'char_rise',
 'm_walk_test',
 'stair_climbing',
 'side_plank',
 'char_rise_baseline',
 'm_walk_test_baseline',
 'stair_climbing_baseline',
 'side_plank_baseline',
 'char_rise_1yr',
 'm_walk_test_1yr',
 'stair_climbing_1yr',
 'side_plank_1yr',
 'char_rise_2yr',
 'm_walk_test_2yr',
 'stair_climbing_2yr',
 'side_plank_2yr',
 't1rho_mean_fem_whole_cartilage',
 't2_mean_fem_whole_cartilage',
 't1rho_mean_ace_whole_cartilage',
 't2_mean_ace_whole_cartilage',
 't1rho_mean_fem_whole_cartilage_baseline',
 't2_mean_fem_whole_cartilage_baseline',
 't1rho_mean_ace_whole_cartilage_baseline',
 't2_mean_a

In [104]:
df.isna().sum()

participant_id                             0
age                                        0
gender                                     0
height_m                                   0
weight_kg                                  0
bmi                                        0
mhi5_score                                 0
mhi5_score_baseline                        0
hoos_symptoms_score                        0
hoos_adl_score                             0
hoos_sports_score                          0
hoos_qol_score                             0
hoos_womac_pain                            0
hoos_womac_stiffness                       0
hoos_womac_function                        0
hoos_pain_score                            0
hoos_pain_score_baseline_arm_1             0
hoos_pain_score_1_yr_follow_up_arm_1       0
hoos_pain_score_2_yr_follow_up_arm_1       0
char_rise                                  0
m_walk_test                                0
stair_climbing                             0
side_plank

In [105]:
#df.dropna(inplace=True)
#to get rid of some columns w/ NA df = df.dropna(subset=['colA', 'colC']) 
df.describe()

,age,gender,height_m,weight_kg,bmi,mhi5_score,mhi5_score_baseline,hoos_symptoms_score,hoos_adl_score,hoos_sports_score,hoos_qol_score,hoos_womac_pain,hoos_womac_stiffness,hoos_womac_function,hoos_pain_score,hoos_pain_score_baseline_arm_1,hoos_pain_score_1_yr_follow_up_arm_1,hoos_pain_score_2_yr_follow_up_arm_1,char_rise,m_walk_test,stair_climbing,side_plank,char_rise_baseline,m_walk_test_baseline,stair_climbing_baseline,side_plank_baseline,char_rise_1yr,m_walk_test_1yr,stair_climbing_1yr,side_plank_1yr,char_rise_2yr,m_walk_test_2yr,stair_climbing_2yr,side_plank_2yr,t1rho_mean_fem_whole_cartilage,t2_mean_fem_whole_cartilage,t1rho_mean_ace_whole_cartilage,t2_mean_ace_whole_cartilage,t1rho_mean_fem_whole_cartilage_baseline,t2_mean_fem_whole_cartilage_baseline,t1rho_mean_ace_whole_cartilage_baseline,t2_mean_ace_whole_cartilage_baseline,race___1,race___2,race___3,race___4,race___5,race___6,ethnicity___1,ethnicity___2,ethnicity___3
count,28.000000,28.000000,28.000000,28.000000,28.000000,28.000000,28.000000,28.000000,28.000000,28.000000,28.000000,28.000000,28.000000,28.000000,28.000000,28.000000,28.000000,28.000000,28.000000,28.000000,28.000000,28.000000,28.000000,28.000000,28.000000,28.000000,28.000000,28.000000,28.000000,28.000000,28.000000,28.000000,28.000000,28.000000,28.000000,28.000000,28.000000,28.000000,28.000000,28.000000,28.000000,28.000000,28.0,28.0,28.0,28.0,28.0,28.0,28.0,28.0,28.0
mean,56.142857,1.607143,1.667357,66.460714,23.585000,79.428571,85.142857,91.964286,93.539643,91.294643,91.071429,95.000000,91.964286,93.224643,93.839286,93.392857,92.232143,94.642857,20.214286,20.374679,10.930357,85.081071,21.642857,18.522857,9.632143,87.704286,23.750000,18.387857,9.982857,83.987143,23.714286,19.284286,10.490714,85.991786,35.709459,34.322913,37.457646,33.854724,35.095976,30.630525,34.216646,28.660837,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
std,11.127924,0.497347,0.114125,16.457508,2.990344,14.835964,8.352042,10.744446,13.204678,16.167820,10.815767,10.886621,13.269939,13.901774,11.476267,13.091811,15.976535,11.660996,7.057211,4.093645,2.449444,50.062005,5.451566,2.662701,1.575306,34.981300,5.680539,2.816037,2.326358,33.508928,6.258962,3.865233,2.001575,42.206654,4.133918,5.446550,3.175347,3.503544,2.005910,4.214153,2.762906,4.140906,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
min,39.000000,1.000000,1.475000,47.400000,19.110000,44.000000,72.000000,65.000000,50.000000,50.000000,75.000000,55.000000,50.000000,47.060000,57.500000,50.000000,37.500000,57.500000,1.000000,7.430000,7.430000,4.930000,12.000000,14.510000,7.350000,27.000000,16.000000,14.500000,6.900000,42.800000,12.000000,14.200000,7.800000,25.930000,30.462926,26.898051,32.053982,27.253391,30.522739,24.029906,27.628239,15.525839,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
25%,43.000000,1.000000,1.585000,54.560000,21.790000,72.000000,80.000000,88.750000,97.795000,93.750000,79.687500,98.750000,84.375000,97.795000,96.875000,94.375000,94.375000,99.375000,18.000000,18.190000,9.605000,56.375000,18.750000,17.000000,8.310000,58.720000,19.000000,15.775000,8.100000,62.205000,18.000000,15.500000,9.400000,56.380000,32.258529,29.948468,35.564938,31.542748,33.475453,27.264221,32.610985,27.043071,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
50%,60.500000,2.000000,1.660000,62.865000,23.030000,84.000000,86.000000,95.000000,100.000000,100.000000,96.875000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,20.500000,20.760000,10.810000,79.760000,23.000000,17.875000,9.505000,88.050000,23.000000,18.855000,9.505000,75.455000,24.500000,19.750000,10.250000,76.000000,35.012596,33.566438,37.455957,33.288479,35.160354,30.225600,33.898055,29.043655,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
75%,64.000000,2.000000,1.720000,69.920000,24.080000,92.000000,92.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,24.000000,22.810000,11.290000,103.162500,24.250000,20.550000,10.330000,104.125000,28.250000,20.400000,11.270000,92.500000,27.000000,2

In [106]:

y = df['hoos_pain_score']

In [77]:
#multioutput
y = df[['hoos_pain_score', 't1rho_mean_fem_whole_cartilage',
 't2_mean_fem_whole_cartilage', 't1rho_mean_ace_whole_cartilage',
 't2_mean_ace_whole_cartilage']]

In [107]:
df.drop(['participant_id','hoos_pain_score', 'hoos_symptoms_score','hoos_adl_score',
         'hoos_sports_score','hoos_qol_score','hoos_womac_pain','hoos_womac_stiffness','hoos_womac_function',
         't1rho_mean_fem_whole_cartilage',
 't2_mean_fem_whole_cartilage',
 't1rho_mean_ace_whole_cartilage',
 't2_mean_ace_whole_cartilage',
 't1rho_mean_fem_whole_cartilage_baseline',
 't2_mean_fem_whole_cartilage_baseline',
 't1rho_mean_ace_whole_cartilage_baseline',
 't2_mean_ace_whole_cartilage_baseline','race___1',
 'race___2',
 'race___3',
 'race___4',
 'race___5',
 'race___6',
 'ethnicity___1',
 'ethnicity___2',
 'ethnicity___3'], axis=1, inplace=True)


In [108]:
list(df.columns)

['age',
 'gender',
 'height_m',
 'weight_kg',
 'bmi',
 'mhi5_score',
 'mhi5_score_baseline',
 'hoos_pain_score_baseline_arm_1',
 'hoos_pain_score_1_yr_follow_up_arm_1',
 'hoos_pain_score_2_yr_follow_up_arm_1',
 'char_rise',
 'm_walk_test',
 'stair_climbing',
 'side_plank',
 'char_rise_baseline',
 'm_walk_test_baseline',
 'stair_climbing_baseline',
 'side_plank_baseline',
 'char_rise_1yr',
 'm_walk_test_1yr',
 'stair_climbing_1yr',
 'side_plank_1yr',
 'char_rise_2yr',
 'm_walk_test_2yr',
 'stair_climbing_2yr',
 'side_plank_2yr']

In [109]:
#run rf classifier base model on full dataset
clf = ExtraTreesRegressor(n_estimators=1000, max_depth=None,
    min_samples_split=2, random_state=0, max_features = "auto")
clf.fit(df, y)

ExtraTreesRegressor(n_estimators=1000, random_state=0)

In [110]:
#examine feature importance
feature_df = pd.DataFrame({'feature': df.columns,
                          'importance': clf.feature_importances_})
feature_df.sort_values(by='importance', ascending=False)

,feature,importance
9,hoos_pain_score_2_yr_follow_up_arm_1,0.497702
8,hoos_pain_score_1_yr_follow_up_arm_1,0.174639
10,char_rise,0.047432
11,m_walk_test,0.041390
7,hoos_pain_score_baseline_arm_1,0.040185
21,side_plank_1yr,0.036155
25,side_plank_2yr,0.035833
13,side_plank,0.028543
17,side_plank_baseline,0.025723
5,mhi5_score,0.022495


In [111]:
# Split up training and testing Sets
from sklearn.model_selection import train_test_split

train_features, test_features, train_labels, test_labels = train_test_split(df, y, 
                                                                            test_size = 0.4, random_state = 42)
print('Training Features Shape:', train_features.shape)
print('Training Labels Shape:', train_labels.shape)
print('Testing Features Shape:', test_features.shape)
print('Testing Labels Shape:', test_labels.shape)

Training Features Shape: (16, 26)
Training Labels Shape: (16,)
Testing Features Shape: (12, 26)
Testing Labels Shape: (12,)


In [112]:
print('Parameters currently in use:\n')
pprint(clf.get_params())

Parameters currently in use:

{'bootstrap': False,
 'ccp_alpha': 0.0,
 'criterion': 'squared_error',
 'max_depth': None,
 'max_features': 'auto',
 'max_leaf_nodes': None,
 'max_samples': None,
 'min_impurity_decrease': 0.0,
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'min_weight_fraction_leaf': 0.0,
 'n_estimators': 1000,
 'n_jobs': None,
 'oob_score': False,
 'random_state': 0,
 'verbose': 0,
 'warm_start': False}


In [113]:
#Create a random grid to search for best hyperparameters
# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 200, stop = 2000, num = 10)]
# Number of features to consider at every split
max_features = ['auto', 'sqrt']
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
max_depth.append(None)
# Minimum number of samples required to split a node
min_samples_split = [2, 5, 10]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 4]
# Method of selecting samples for training each tree
bootstrap = [True, False]

# Create the random grid
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}

pprint(random_grid)

{'bootstrap': [True, False],
 'max_depth': [10, 20, 30, 40, 50, 60, 70, 80, 90, 100, 110, None],
 'max_features': ['auto', 'sqrt'],
 'min_samples_leaf': [1, 2, 4],
 'min_samples_split': [2, 5, 10],
 'n_estimators': [200, 400, 600, 800, 1000, 1200, 1400, 1600, 1800, 2000]}


In [114]:
rf = ExtraTreesRegressor() 
rf_random = RandomizedSearchCV(estimator=rf, param_distributions=random_grid,
                              n_iter = 100, scoring='neg_mean_absolute_error', 
                              cv = 3, verbose=2, n_jobs=-1,
                              return_train_score=True)


rf_random.fit(train_features, train_labels);

Fitting 3 folds for each of 100 candidates, totalling 300 fits


In [115]:
rf_random.best_params_

{'n_estimators': 600,
 'min_samples_split': 5,
 'min_samples_leaf': 1,
 'max_features': 'auto',
 'max_depth': 40,
 'bootstrap': False}

In [116]:
#evaluate accuracy of base model using .score 
base_model = ExtraTreesRegressor(n_estimators = 10, random_state = 42)
base_model.fit(train_features, train_labels)
base_model.score(test_features, test_labels) #r squared

0.545886736214605

In [117]:
# Make predictions on test data using the model trained on original data
predictions = base_model.predict(test_features)
errors = abs(predictions - test_labels)
# Performance metrics

print('Metrics for Random Forest Trained on Original Data')
print('Average absolute error:', round(np.mean(errors), 2), 'points.')
# Calculate mean absolute percentage error (MAPE)
mape = 100 * (errors / test_labels)
# Calculate and display accuracy
accuracy = 100 - np.mean(mape)
print('Accuracy:', round(accuracy, 2), '%.')

Metrics for Random Forest Trained on Original Data
Average absolute error: 4.56 points.
Accuracy: 94.38 %.


In [118]:
#evaluate accuracy of model using evaluate function
def evaluate(model, test_features, test_labels):
    predictions = model.predict(test_features)
    errors = abs(predictions - test_labels)
    mape = 100 * np.mean(errors / test_labels)
    accuracy = 100 - mape
    print('Model Performance')
    print('Average Error: {:0.4f} degrees.'.format(np.mean(errors)))
    print('Accuracy = {:0.2f}%.'.format(accuracy))
    
    return accuracy

In [119]:
base_accuracy = evaluate(base_model, test_features, test_labels)

Model Performance
Average Error: 4.5625 degrees.
Accuracy = 94.38%.


In [120]:
#evaluation accuracy of best model from random grid search
best_random = rf_random.best_estimator_
best_random.score(test_features, test_labels) #r squared

0.6185378408400968

In [121]:
# Make predictions on test data using the model trained on original data
predictions = best_random.predict(test_features)
errors = abs(predictions - test_labels)
# Performance metrics

print('Metrics for Random Forest Trained on Original Data')
print('Average absolute error:', round(np.mean(errors), 2), 'points.')
# Calculate mean absolute percentage error (MAPE)
mape = 100 * (errors / test_labels)
# Calculate and display accuracy
accuracy = 100 - np.mean(mape)
print('Accuracy:', round(accuracy, 2), '%.')

Metrics for Random Forest Trained on Original Data
Average absolute error: 3.76 points.
Accuracy: 95.37 %.


In [122]:
#evaluate accuracy of model using accuracy_score 
random_accuracy = evaluate(best_random, test_features, test_labels)

Model Performance
Average Error: 3.7628 degrees.
Accuracy = 95.37%.


In [123]:
#examine feature importances from best model from random grid search
feature_df = pd.DataFrame({'feature': df.columns,
                          'importance': best_random.feature_importances_})
feature_df.sort_values(by='importance', ascending=False)

,feature,importance
9,hoos_pain_score_2_yr_follow_up_arm_1,0.671352
8,hoos_pain_score_1_yr_follow_up_arm_1,0.212591
7,hoos_pain_score_baseline_arm_1,0.028945
10,char_rise,0.023087
5,mhi5_score,0.020526
25,side_plank_2yr,0.012713
11,m_walk_test,0.007267
17,side_plank_baseline,0.005344
13,side_plank,0.005138
19,m_walk_test_1yr,0.004801


In [ ]:
#https://scikit-learn.org/stable/modules/multiclass.html#multioutput-regression

In [124]:
viz = dtreeviz(best_random.estimators_[0], train_labels, train_features)



AttributeError: 'DataFrame' object has no attribute 'dtype'

[CV] END bootstrap=True, max_depth=100, max_features=sqrt, min_samples_leaf=2, min_samples_split=5, n_estimators=1800; total time=   4.8s
[CV] END bootstrap=False, max_depth=30, max_features=auto, min_samples_leaf=4, min_samples_split=10, n_estimators=2000; total time=   3.8s
[CV] END bootstrap=False, max_depth=80, max_features=auto, min_samples_leaf=1, min_samples_split=5, n_estimators=400; total time=   0.8s
[CV] END bootstrap=False, max_depth=80, max_features=auto, min_samples_leaf=2, min_samples_split=5, n_estimators=200; total time=   0.5s
[CV] END bootstrap=True, max_depth=60, max_features=auto, min_samples_leaf=2, min_samples_split=10, n_estimators=400; total time=   1.0s
[CV] END bootstrap=True, max_depth=80, max_features=sqrt, min_samples_leaf=4, min_samples_split=5, n_estimators=400; total time=   1.1s
[CV] END bootstrap=False, max_depth=100, max_features=sqrt, min_samples_leaf=4, min_samples_split=5, n_estimators=1800; total time=   3.5s
[CV] END bootstrap=False, max_depth=5